
# Mean/Median vs. Simulation — A Hands‑On Mini‑Lab (Colab Ready)

**What you'll learn**  
1) When the **mean** and **median** tell different stories (skewed data, outliers).  
2) How a simple **Monte Carlo simulation** reveals the full distribution of outcomes (not just an average).

> Tip: In Google Colab, go to **Runtime → Run all**.



## Learning goals
- Understand why the mean can be misleading under **skew** and **outliers**—and why the median can be more robust.
- Build intuition for **uncertainty** using a simple **project timeline** simulation.
- Answer questions like: *What's the chance we miss a deadline?* or *What's the 95th percentile worst-case?*

---

## Setup
Run this cell to import libraries and set a seed for reproducibility.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

def show_stats(arr, label="Data"):
    arr = np.asarray(arr)
    print(f"{label} — n={len(arr):,}")
    print(f"  Mean   : {np.mean(arr):.3f}")
    print(f"  Median : {np.median(arr):.3f}")
    print(f"  Std Dev: {np.std(arr, ddof=1):.3f}")



## Part A — Mean vs. Median on a Skewed Distribution

We'll start with a **lognormal** distribution, a classic example of positive skew
(think incomes, waiting times). We'll compare the **mean** and **median** and see how
they react to **outliers**.


In [ ]:

# Generate a positively skewed dataset
n = 10_000
mu, sigma = 1.0, 0.9      # lognormal parameters
data = np.random.lognormal(mean=mu, sigma=sigma, size=n)

show_stats(data, "Skewed data")

# Plot histogram
plt.figure(figsize=(8,4.5))
plt.hist(data, bins=60)  # default colors, single chart
plt.title("Skewed Data (Lognormal) — Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

# Add a few extreme outliers and compare
outliers = np.array([200, 300, 500, 800, 1200])  # a handful of huge values
data_with_outliers = np.concatenate([data, outliers])

print("\nWith a few extreme outliers added:")
show_stats(data_with_outliers, "Skewed data + outliers")

plt.figure(figsize=(8,4.5))
plt.hist(data_with_outliers, bins=80)
plt.title("Skewed Data + Outliers — Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

print("\nObservation:")
print("- The MEAN jumps a lot after adding a few massive values.")
print("- The MEDIAN moves much less — it's more robust to outliers.")



## Part B — Monte Carlo Simulation for a Project Timeline

Imagine a 3‑task project. Each task has an **optimistic**, **most likely**, and **pessimistic**
duration (PERT/triangular‑style thinking). Instead of summing averages, we’ll **simulate**
many possible timelines to see the **distribution** of total duration.

We'll answer:
- What's the **mean** and **median** completion time?
- What's the **90th** and **95th percentile** (risk tail)?
- What's the probability we **finish within a deadline**?


In [ ]:

# Helper: sample from a triangular distribution (optimistic, most_likely, pessimistic)
def rtriangular(low, mode, high, size):
    return np.random.triangular(left=low, mode=mode, right=high, size=size)

def simulate_project(tasks, n_sims=50_000, deadline=None):
    """
    tasks: list of dicts with keys {'name','optimistic','likely','pessimistic'}
    deadline: numeric (days) or None
    Returns dict with 'totals' array and summary stats.
    """
    totals = np.zeros(n_sims)
    for t in tasks:
        durations = rtriangular(t['optimistic'], t['likely'], t['pessimistic'], n_sims)
        totals += durations

    stats = {
        "mean": float(np.mean(totals)),
        "median": float(np.median(totals)),
        "p90": float(np.percentile(totals, 90)),
        "p95": float(np.percentile(totals, 95)),
    }
    if deadline is not None:
        prob_on_time = float(np.mean(totals <= deadline))
        stats["deadline"] = float(deadline)
        stats["prob_on_time"] = prob_on_time
    return {"totals": totals, "stats": stats}

# Define a 3‑task project
tasks = [
    {"name": "Task A", "optimistic": 3, "likely": 5, "pessimistic": 10},
    {"name": "Task B", "optimistic": 2, "likely": 4, "pessimistic": 7},
    {"name": "Task C", "optimistic": 4, "likely": 6, "pessimistic": 12},
]

deadline_days = 20  # try changing this

result = simulate_project(tasks, n_sims=60_000, deadline=deadline_days)
stats = result["stats"]
totals = result["totals"]

print("Simulation summary (days):")
for k in ["mean", "median", "p90", "p95"]:
    print(f"  {k.upper():<6}: {stats[k]:.2f}")
if "deadline" in stats:
    print(f"  DEADLINE : {stats['deadline']:.2f}")
    print(f"  P(on time): {100*stats['prob_on_time']:.1f}%")

plt.figure(figsize=(8,4.5))
plt.hist(totals, bins=60)
plt.axvline(stats["median"], linestyle="--", label="Median")
plt.axvline(stats["mean"], linestyle=":", label="Mean")
if "deadline" in stats:
    plt.axvline(stats["deadline"], linestyle="-.", label="Deadline")
plt.title("Project Completion Time — Monte Carlo Distribution")
plt.xlabel("Total days")
plt.ylabel("Frequency")
plt.legend()
plt.show()

print("\nWhy simulate?")
print("- You see the *whole distribution*, not just an average.")
print("- You can read probabilities from it (e.g., on-time chance).")
print("- Percentiles (P90, P95) help plan buffers and risk.")



## Part C — What‑If Playground

Tweak task ranges or the deadline and re‑run to see how the distribution changes.


In [ ]:

# Try your own scenario here:
your_tasks = [
    {"name": "Analysis", "optimistic": 1, "likely": 2, "pessimistic": 5},
    {"name": "Build",    "optimistic": 3, "likely": 5, "pessimistic": 9},
    {"name": "Test",     "optimistic": 2, "likely": 3, "pessimistic": 6},
    {"name": "Deploy",   "optimistic": 1, "likely": 2, "pessimistic": 4},
]

your_deadline = 18  # change me

res = simulate_project(your_tasks, n_sims=50_000, deadline=your_deadline)
s = res["stats"]
t = res["totals"]

print("Your scenario — summary (days):")
for k in ["mean", "median", "p90", "p95"]:
    print(f"  {k.upper():<6}: {s[k]:.2f}")
print(f"  DEADLINE : {s['deadline']:.2f}")
print(f"  P(on time): {100*s['prob_on_time']:.1f}%")

plt.figure(figsize=(8,4.5))
plt.hist(t, bins=60)
plt.axvline(s["median"], linestyle="--", label="Median")
plt.axvline(s["mean"], linestyle=":", label="Mean")
plt.axvline(s["deadline"], linestyle="-.", label="Deadline")
plt.title("Your Scenario — Monte Carlo Distribution")
plt.xlabel("Total days")
plt.ylabel("Frequency")
plt.legend()
plt.show()



## Wrap‑up (Cheat Sheet)

- **Mean vs. Median**  
  - Mean is **sensitive** to extreme values; median is **robust**.  
  - In skewed data, the median often represents the **typical** value better.

- **Why simulate (Monte Carlo)?**  
  - Get the **full distribution** of outcomes.  
  - Compute **probabilities** (e.g., P(finish ≤ 20 days)).  
  - Plan with **percentiles** (P90/P95) and see **risk** clearly.

**Next step idea:** Add costs or dependencies, then simulate cost‑and‑schedule together.
